## Schema SAIDs

A key feature of KERI and ACDCs is the use of SAIDs (Self-Addressing Identifiers) for schemas. The SAID in a schema's `$id` field is a hash of the canonical form of that schema block.

* **Why it matters:**
    * **Integrity:** If anyone modifies the schema file after its SAID has been calculated and embedded, the SAID will no longer match the content, making tampering evident.
    * **Immutability:** Once a schema version is SAIDified and published, it's cryptographically locked. New versions require a new SAID.
    * **Lookup:** SAIDs provide a universal, unique identifier to retrieve a specific, verified version of a schema.
      
Calculating and embedding these SAIDs requires a specific process, often called **"SAIDifying"**. This involves calculating the SAIDs for the innermost blocks (like attributes, edges, rules) first, embedding them, and then calculating the SAID for the next level up, until the top-level schema SAID is computed.

## Practical: SAIDifying a Schema with KLI

Let's create a very simple schema and process it using the `kli` tool.

### Step 1: Write the Schema JSON

First, create a JSON file (e.g., `sample_credential_schema.json`) with the basic structure. Notice the `$id` fields are initially empty strings `""` – they will be filled in by `kli`.

### Step 2: SAIDify the Schema

Now, we need to process the schema to embed the Self-Addressing Identifiers (SAIDs). For this, we have a convenient [Python script](scripts/saidify.py) available in the scripts folder. We will use this script, which calculates the required digests from the content and adds the SAIDs to the file. 

In [1]:
from scripts.saidify import process_schema_file

# Run the saidify script
process_schema_file("./config/schemas/sample_schema.bak.json", "./config/schemas/sample_schema.json", True)

# Displays all "$id" values from any JSON objects found recursively in the schema file
print("\ncalculated saids ($id):")
!jq '.. | objects | .["$id"]? // empty' ./config/schemas/sample_schema.json


Successfully wrote processed data to ./config/schemas/sample_schema.json

calculated saids ($id):
"EEARD7SAMt2xhHfcWLZW7UoGGmQtMfsmBN9cOUGO0y44"
"ED614TseulOlXWhFNsOcKIKt9Na0gCByugqyKVsva-gl"


After running this command, if you inspect simple_credential_schema.json again, you will see that the previously empty `"$id": ""` fields (both the top-level one and the one inside the a block) have been populated with SAID strings (long Base64-like identifiers).

For example, the top-level `$id` might now look something like:
`"$id": "EABC123abc..."` 
And the $id inside the a block might look like:
`"$id": "EFGH456def..."`

(Note: The actual SAIDs generated will depend on the exact content and formatting used by the kli saidify tool at the time of execution.)

You now have a cryptographically verifiable schema identified by its top-level SAID!

<div class="alert alert-info">
  <b>💡 TIP</b><hr>
    The KERI command-line tool (`kli`) provides the <code>kli saidify</code> command. When used like <code>kli saidify --file &lt;filename&gt; --label '&lt;label&gt;'</code>, it calculates a SAID for the specified file content and can embed it into the field matching the label (e.g., <code>"$id"</code> at the top level).
    <br><br>
    However, automatically processing nested structures and dependencies within complex ACDC schemas typically requires helper tools (like KASLCred) or custom scripts to ensure all inner SAIDs are calculated and embedded correctly before the final outer SAID is generated.
</div>

### Step 3: Caching Schemas (Conceptual) 🚧
For an issuer to issue an ACDC using this schema, and for a recipient/verifier to validate it, they both need access to this exact, SAIDified schema definition.

How do they get it? The SAID acts as a universal lookup key. Common ways to make schemas available include:

- Simple Web Server: Host the SAIDified JSON file on a basic web server (like the vLEI-server mentioned in the tutorial ). Controllers can be configured (often via OOBIs, covered later) to fetch the schema from that URL using its SAID.   
- Content-Addressable Network: Store the schema on a network like IPFS, where the SAID naturally aligns with the content hash used for retrieval.
- Direct Exchange: For specific interactions, the schema could potentially be exchanged directly between parties (though less common for widely used schemas).
The key point is that the schema, identified by its SAID, must be retrievable by parties needing to issue or verify credentials based on it.

In the next notebook, we'll use our SAIDified schema to set up a Credential Registry and issue our first actual ACDC.